In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Load CSV data
data = pd.read_csv('C:/Users/rsaad/Downloads/New folder/hourly_BTC.csv')

# Step 1: Calculate trading range and volatility
data['range'] = data['high'] - data['low']
data['volatility'] = data['range'].rolling(window=20).std()

In [3]:
# Step 2: Identify trading ranges (low volatility periods)
threshold_volatility = data['volatility'].quantile(0.3)  # Bottom 30% volatility as range
data['is_range'] = data['volatility'] < threshold_volatility

In [4]:
# Identify range boundaries
data['range_high'] = np.nan
data['range_low'] = np.nan
in_range = False
range_start = 0
for i in range(len(data)):
    if data['is_range'].iloc[i] and not in_range:
        in_range = True
        range_start = i
    elif not data['is_range'].iloc[i] and in_range:
        in_range = False
        range_high = data['high'].iloc[range_start:i].max()
        range_low = data['low'].iloc[range_start:i].min()
        data.loc[range_start:i-1, 'range_high'] = range_high
        data.loc[range_start:i-1, 'range_low'] = range_low

In [5]:
# Step 3: Detect trends (Higher Highs/Higher Lows or Lower Highs/Lower Lows)
data['hh'] = (data['high'] > data['high'].shift(1)) & (data['high'].shift(1) > data['high'].shift(2))
data['hl'] = (data['low'] > data['low'].shift(1)) & (data['low'].shift(1) > data['low'].shift(2))
data['lh'] = (data['high'] < data['high'].shift(1)) & (data['high'].shift(1) < data['high'].shift(2))
data['ll'] = (data['low'] < data['low'].shift(1)) & (data['low'].shift(1) < data['low'].shift(2))

In [6]:
# Classify trend: 1 for bullish (HH/HL), -1 for bearish (LH/LL), 0 for ranging
data['trend'] = 0
data.loc[data['hh'] & data['hl'], 'trend'] = 1
data.loc[data['lh'] & data['ll'], 'trend'] = -1

In [7]:
# Step 4: Binary classification for breakout prediction
# Features: range size, volume, volatility
data['range_size'] = data['range_high'] - data['range_low']
data['breakout'] = 0
for i in range(len(data)):
    if i < len(data) - 1 and data['is_range'].iloc[i]:
        next_high = data['high'].iloc[i+1]
        next_low = data['low'].iloc[i+1]
        if not pd.isna(data['range_high'].iloc[i]):
            if next_high > data['range_high'].iloc[i] or next_low < data['range_low'].iloc[i]:
                data.loc[i, 'breakout'] = 1

In [8]:
# Prepare data for classification
features = ['range_size', 'volume', 'volatility']
X = data[features].dropna()
y = data['breakout'].loc[X.index]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [ ]:
# Predict breakouts
data['breakout_pred'] = np.nan
X_full = scaler.transform(data[features].dropna())
data.loc[data[features].dropna().index, 'breakout_pred'] = model.predict(X_full)

# Step 5: Plotting
plt.figure(figsize=(14, 7))
plt.plot(data['date'], data['close'], label='Close Price', color='blue')

# Plot ranges
range_periods = data[data['is_range']]
for i in range(len(range_periods)):
    if not pd.isna(range_periods['range_high'].iloc[i]):
        plt.axvspan(range_periods['date'].iloc[i], range_periods['date'].iloc[i], 
                    color='gray', alpha=0.3)


Error in callback <function _draw_all_if_interactive at 0x000001CE8A4A9DA0> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

In [ ]:
# Plot trends
bullish = data[data['trend'] == 1]
bearish = data[data['trend'] == -1]
plt.scatter(bullish['date'], bullish['close'], color='green', marker='^', label='Bullish Trend')
plt.scatter(bearish['date'], bearish['close'], color='red', marker='v', label='Bearish Trend')

In [ ]:
# Plot breakout predictions
breakouts = data[data['breakout_pred'] == 1]
plt.scatter(breakouts['date'], breakouts['close'], color='orange', marker='o', label='Predicted Breakout')

plt.title('Price Movement with Ranges, Trends, and Breakout Predictions')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()


In [ ]:
# Save the plot
plt.savefig('range_trend_analysis.png')

# Save results to CSV
data.to_csv('analyzed_market_data.csv', index=False)

# Print summary
print("Number of identified ranges:", data['is_range'].sum())
print("Number of bullish trends:", (data['trend'] == 1).sum())
print("Number of bearish trends:", (data['trend'] == -1).sum())
print("Number of predicted breakouts:", data['breakout_pred'].sum())